In [ ]:
;pwd

In [ ]:
]activate ..

In [ ]:
using Apophis

In [ ]:
# using TaylorIntegration, LinearAlgebra # Apophis.jl @reexports TaylorIntegration and LinearAlgebra
using Plots, JLD, DelimitedFiles, Dates
using Statistics: mean, std
using AstroTime

In [ ]:
# recover integration from .jld file
vars = ["tv_jpl_integ", "xv1", "tvS1", "xvS1", "gvS1"] #names of variables
filename = string("Apophis_jt.jld")
for i in eachindex(vars)
    ex = Symbol(vars[i])
    @eval $ex = load($filename, vars[$i])
end

In [ ]:
sort(union(apophisdofs, ssdofs)) == collect(1:72)

xv1[2,apophisdofs]

xv1[2,sundofs]

xv1[20,ssdofs[end-6:end]]

In [ ]:
jpl_radar = readdlm("../Apophis_JPL_data.dat", '\t')

findall(x -> x == "Hz", jpl_radar[:,5])

findall(x -> x == "us", jpl_radar[:,5])

hola1 = jpl_radar[:,5] .== "Hz"
hola2 = jpl_radar[:,5] .== "us"
hola3 = hola1 .| hola2
hola4 = hola1 .& hola2
all(hola3), !any(hola4)

jpl_radar[:,2]

In [ ]:
#construct vector of observation times
#####`union` removes repeated elements
#NOTE: resulting vector does not include t0
# tv_jpl = union( Dates.datetime2julian.(  DateTime.(jpl_radar[:,2], "y-m-d H:M:S")) );
df_jpl = "y-m-d H:M:S"
tv_jpl = Dates.datetime2julian.(  DateTime.(jpl_radar[:,2], df_jpl));

In [ ]:
#extract transmitter frequencies (Hz)
transmitter_freq = jpl_radar[:,6]*1e6;

In [ ]:
#monostatic mode: check that each receiver and transmitter are the same
all( jpl_radar[:,7] .== jpl_radar[:,8] )

In [ ]:
#then, get the station codes
station_codes = Int.(jpl_radar[:,7]);

In [ ]:
#check that time vectors from NEODyS and JPL are the same
#tv_neodys_obs == union(t0, tv_jpl[tv_jpl .> t0])
tv_jpl_utc = DateTime.(jpl_radar[:,2], df_jpl)[8:end]
tv_jpl_utc_julian = Dates.datetime2julian.(tv_jpl_utc)
tv_jpl_utc_julian_unrepeated = union(julian(UTCEpoch(TDBEpoch(t0, origin=:julian))).Δt, tv_jpl_utc_julian)

In [ ]:
#extract time-delay and Doppler-shift measurements
#will filter those afterwards by type of measurement
deldop_jpl = jpl_radar[:, 3];

In [ ]:
#get indices of delay observations
del_ind = findall(x->x == "us", jpl_radar[:,5])
#get indices of Doppler observations
dop_ind = findall(x->x == "Hz", jpl_radar[:,5]);

In [ ]:
#get index vector of observation times
#takes into account only times in `tv_jpl_integ` greater than t0
radar_jpl_obs_ind = map(y->findfirst(x->x == y, tv_jpl), tv_jpl_utc_julian_unrepeated[2:end])

In [ ]:
tv_jpl[radar_jpl_obs_ind]

In [ ]:
#check that we got `radar_obs_ind` right
tv_jpl[radar_jpl_obs_ind] == tv_jpl_utc_julian_unrepeated[2:end]

In [ ]:
size(radar_jpl_obs_ind)

In [ ]:
size(xv1[2:end,:])

In [ ]:
size(tv_jpl_integ)

In [ ]:
size(station_codes[radar_jpl_obs_ind])

Let $\vec R$ denote the geocentric position of the observing station, $\vec \rho$ the topocentric position of the asteroid relative to the same station,  $\vec r_\mathrm{a}$ the geocentric position of the asteroid and $\vec r_\mathrm{E}$ the geocentric position of the asteroid. Then we have

$$
\vec r_\mathrm{a} =\vec r_\mathrm{E} + \vec R + \vec \rho
$$

Thus, the instantaneous topocentric range $\rho = |\vec \rho|$ of the asteroid may be computed as

$$
\rho = |\vec r_\mathrm{a} - \vec r_\mathrm{E} - \vec R| = \sqrt{(x_\mathrm{a} - x_\mathrm{E} - X)^2+(y_\mathrm{a} - y_\mathrm{E} - Y)^2+(z_\mathrm{a} - z_\mathrm{E} - Z)^2}
$$

And the instantaneous range rate $\dot \rho$ may be computed as:

$$
\dot \rho = \frac{1}{\rho}\vec{\rho} \cdot \dot{\vec{\rho}}
$$

dop_neodys_obs_ind = findall(x->in(x, jd_dop), tv_jpl_integ);
del_neodys_obs_ind = findall(x->in(x, jd_del), tv_jpl_integ)

#check that all elements in jd_dop/jd_del were found in tv_jpl_integ
@show length(dop_neodys_obs_ind) == length(jd_dop)
@show length(del_neodys_obs_ind) == length(jd_del)

In [ ]:
const ea = 4 #Earth's index
const N = 12 

# Apophis geocentric range, radial velocity
r(x) = sqrt( (x[3N-2]-x[3ea-2])^2+(x[3N-1]-x[3ea-1])^2+(x[3N]-x[3ea])^2 )
vr(x) = ( (x[3N-2]-x[3ea-2])*(x[6N-2]-x[3(N+ea)-2])+(x[3N-1]-x[3ea-1])*(x[6N-1]-x[3(N+ea)-1])+(x[3N]-x[3ea])*(x[6N]-x[3(N+ea)]) )/r(x)

# Apophis topocentric range, radial velocity
function r_topo(x, station_code, jd_utc)
    r_a = [x[3N-2], x[3N-1], x[3N]] # asteroid barycentric position (au)
    r_E = [x[3ea-2], x[3ea-1], x[3ea]] # Earth barycentric position (au)
    R_station = observer_position(station_code, jd_utc)/au # station geocentric position (au)
    rho_vec = r_a - r_E - R_station
    return sqrt( rho_vec[1]^2+rho_vec[2]^2+rho_vec[3]^2 )
end

function vr_topo(x, station_code, jd_utc)
    r_a = [x[3N-2], x[3N-1], x[3N]] # asteroid barycentric position (au)
    v_a = [x[6N-2], x[6N-1], x[6N]] # asteroid barycentric velocity (au/day)
    
    r_E = [x[3ea-2], x[3ea-1], x[3ea]] # Earth barycentric position (au)
    v_E = [x[3(N+ea)-2], x[3(N+ea)-1], x[3(N+ea)]] # Earth barycentric velocity (au/day)
    
    R_station_T1 = observer_position(station_code, Taylor1([jd_utc, one(jd_utc)], 1))/au # station geocentric position (Taylor1) (au)
    R_station = [R_station_T1[1][0], R_station_T1[2][0], R_station_T1[3][0]] # station geocentric position (au)
    V_station = [R_station_T1[1][1], R_station_T1[2][1], R_station_T1[3][1]] # station geocentric velocity (au/day)
    
    ρ_vec = r_a - r_E - R_station # \vec \rho
    ρvel_vec = v_a - v_E - V_station # \dot \vec \rho
    ρ = sqrt( ρ_vec[1]^2+ρ_vec[2]^2+ρ_vec[3]^2 )
    
    return dot(ρ_vec, ρvel_vec)/ρ
end

# x: Solar System + Apophis state at receiving time
# station_code: observing station identifier (MPC nomenclature)
# t: time of echo reception (TDB)
# f_T: transmitter frequency
function delay_doppler(x, station_code, t, f_T)
    # Compute Taylor expansion at receiving time (TDB)
    t_r = Taylor1([t, one(t)], Apophis.order)
    x_r = Taylor1.(x, Apophis.order)
    dx_r = similar(x_r)
    @time TaylorIntegration.jetcoeffs!(Val(Apophis.RNp1BP_pN_A_J234E_J2S_ng!), t_r, x_r, dx_r)
    
    r_a = x_r[apophisdofs[1:3]] # asteroid barycentric position (in a.u.) at receiving time (TDB)
    r_E = x_r[(3ea-2):3ea] # Earth barycentric position (in a.u.) at receiving time (TDB)
    # convert Julian date of receiving time from TDB to UTC
    t_r_utc_julian = julian(UTCEpoch(TDBEpoch(t, origin=:julian))).Δt
    t_r_utc_julian_T = Taylor1([t_r_utc_julian, one(t_r_utc_julian)], Apophis.order)
    @show t_r_utc_julian
    @show t_r_utc_julian_T
    R_station = observer_position(station_code, t_r_utc_julian_T)/au # station geocentric position (au)
    r_r = r_E + R_station # barycentric position of receiver (au)
    
    # down-leg iteration
    # τ_D first approximation: Eq. (1) Yeomans et al. (1992)
    ρ_r_0 = r_a()-r_r()
    τ_D = sqrt(ρ_r_0[1]^2 + ρ_r_0[2]^2 + ρ_r_0[3]^2)/c_au_per_day # (days) -R_b/c, but delay is wrt asteroid Center
    # @show τ_D
    for i in 1:5
        # Eq. (3) Yeomans et al. (1992)
        ρ_r = r_a(-τ_D)-r_r()
        # Eq. (4) Yeomans et al. (1992)
        τ_D = sqrt(ρ_r[1]^2 + ρ_r[2]^2 + ρ_r[3]^2)/c_au_per_day # (days) -R_b/c (COM correction) + Δτ_D (relativistic, tropo, iono...)
        # @show τ_D
    end
    # @show τ_D
    # Eq. (2) Yeomans et al. (1992)
    t_b = -τ_D
    
    # up-leg iteration
    # τ_U first estimation: Eq. (5) Yeomans et al. (1992)
    τ_U = τ_D
    # @show τ_U
    for i in 1:6
        # Eq. (7) Yeomans et al. (1992)
        ρ_t = r_a(-τ_D)-r_r(-τ_U-τ_D)
        # Eq. (8) Yeomans et al. (1992)
        τ_U = sqrt(ρ_t[1]^2 + ρ_t[2]^2 + ρ_t[3]^2)/c_au_per_day # (days) -R_b/c (COM correction) + Δτ_U (relativistic, tropo, iono...)
        # @show τ_U
    end
    # @show τ_U
    # Eq. (6) Yeomans et al. (1992)
    t_t = t_b - τ_U
    
    # Eq. (9) Yeomans et al. (1992)
    τ = τ_D + τ_U
    TDB_minus_UTC_r = t - julian(UTCEpoch(TDBEpoch(t, origin=:julian))).Δt
    TDB_minus_UTC_t = (t-τ[0]) - julian(UTCEpoch(TDBEpoch(t-τ[0], origin=:julian))).Δt
    total_time_delay = τ + TDB_minus_UTC_t - TDB_minus_UTC_r
    @show TDB_minus_UTC_t - TDB_minus_UTC_r
    @show total_time_delay

    v_a = x_r[apophisdofs[4:6]]
    v_E = x_r[(3(N+ea)-2):3(N+ea)]
    V_station = differentiate.(R_station)
    v_r = v_E + V_station
    # v_r is equivalent to differentiate.(r_r) except for last order terms
    # @show v_r 
    # @show differentiate.(r_r) # barycentric velocity of receiver (au)
    
    # Eq. (10) Yeomans et al. (1992)
    ρ_vec_dot_t = v_a(t_b)-v_r(t_t)
    ρ_vec_dot_r = v_a(t_b)-v_r()
    
    ρ_vec_t = r_a(t_b) - r_r(t_t)
    ρ_vec_r = r_a(t_b) - r_r()
    
    # Eq. (11) Yeomans et al. (1992)
    ρ_t = sqrt(ρ_vec_t[1]^2+ρ_vec_t[2]^2+ρ_vec_t[3]^2)
    ρ_dot_t = dot(ρ_vec_t, ρ_vec_dot_t)/ρ_t
    ρ_r = sqrt(ρ_vec_r[1]^2+ρ_vec_r[2]^2+ρ_vec_r[3]^2)
    ρ_dot_r = dot(ρ_vec_r, ρ_vec_dot_r)/ρ_r
    
    # @show ρ_dot_t
    # @show ρ_dot_r
    
    # Eq. (12) Yeomans et al. (1992)
    doppler1 = -f_T*(ρ_dot_t+ρ_dot_r)/c_au_per_day
    doppler2 = (ρ_dot_t/ρ_t)*dot(ρ_vec_t, v_r(t_t)) - (ρ_dot_r/ρ_r)*dot(ρ_vec_r, v_r()) - ρ_dot_t*ρ_dot_r
    r_s = x_r[sundofs[1:3]]
    r_ts = r_r(t_t) - r_s(t_t)
    r_rs = r_r() - r_s()
    factor = 1/sqrt(r_ts[1]^2+r_ts[2]^2+r_ts[3]^2) - 1/sqrt(r_rs[1]^2+r_rs[2]^2+r_rs[3]^2)
    doppler3 = μ[1]*factor
    doppler4 = (  dot(v_r(t_t), v_r(t_t)) - dot(v_r(), v_r())  )/2
    doppler_234 = -f_T*(doppler2 + doppler3 + doppler4)/(c_au_per_day^2)
    f_D = doppler1+doppler_234

    return total_time_delay, f_D # total signal delay and Doppler shift
end

In [ ]:
radar_jpl_obs_ind[2]

In [ ]:
tv_jpl[radar_jpl_obs_ind[2]]

In [ ]:
tv_jpl_integ[3]

In [ ]:
tv_jpl[radar_jpl_obs_ind[2]] - tv_jpl_integ[3]

In [ ]:
# tdelay, dshift = delay_doppler(  xv1[3, :], station_codes[radar_jpl_obs_ind[2]], tv_jpl[radar_jpl_obs_ind[2]], transmitter_freq[radar_jpl_obs_ind[2]]  )
tdelay, dshift = delay_doppler(  xv1[3, :], station_codes[radar_jpl_obs_ind[2]], tv_jpl_integ[3], transmitter_freq[radar_jpl_obs_ind[2]]  )

In [ ]:
1e6*86400*tdelay

In [ ]:
102682986.05-1e6*86400*tdelay() #O-C

In [ ]:
dshift

In [ ]:
57880.250 - dshift() #O-C

In [ ]:
tv_jpl_utc_julian_unrepeated[1]

In [ ]:
# get time-delay, Doppler-shift values from integration

tdelay_v1 = Array{Taylor1{Float64}}(undef, length(tv_jpl_integ)-1)
dshift_v1 = Array{Taylor1{Float64}}(undef, length(tv_jpl_integ)-1)

#tv_jpl[radar_obs_ind] == tv_jpl_integ[2:end]
for i in eachindex(tv_jpl_integ[2:end])
    j = radar_jpl_obs_ind[i]
    #@show tv_jpl[j]
    #@show tv_jpl_integ[i]
    @show tv_jpl[j] == tv_jpl_utc_julian_unrepeated[i+1]
    tdelay_v1[i], dshift_v1[i] = delay_doppler(  xv1[i+1, :], station_codes[j], tv_jpl_integ[i+1], transmitter_freq[j]  )
end


# Questions
- Is TDB the actual independent variable in the integration of the Solar System's equations of motion?
- Where can I get the integrated values (evolution?) of the orientation of the Moon, so that I do not have to integrate that from the beginning (1969)?

In [ ]:
#get set of indices in jpl_radar[:,5] which are, resp., delay and Doppler observations (>t0)
del_ind = findall(y->y=="us", jpl_radar[:,5])[3:end]
dop_ind = findall(y->y=="Hz", jpl_radar[:,5])[6:end];

In [ ]:
#get corresponding set of indices in tv_jpl_integ
tv_del_ind = findall(x->in(x, tv_jpl[del_ind]), tv_jpl_utc_julian_unrepeated) .- 1
tv_dop_ind = findall(x->in(x, tv_jpl[dop_ind]), tv_jpl_utc_julian_unrepeated) .- 1;

In [ ]:
#check that indices are correct
union(sort(vcat(tv_del_ind, tv_dop_ind))) == collect(1:29)

In [ ]:
# Observed minus computed (O-C) residuals

# absolute
residual_td = Float64.(jpl_radar[del_ind,3]) - 1e6*86400*tdelay_v1[tv_del_ind] # (usec)
residual_ds = Float64.(jpl_radar[dop_ind,3]) - dshift_v1[tv_dop_ind] # (Hz)

# relative
rel_res_td = residual_td ./ Float64.(jpl_radar[del_ind,3]) # (usec)
rel_res_ds = residual_ds ./ Float64.(jpl_radar[dop_ind,3]); # (Hz)

In [ ]:
scatter(
    tv_jpl_utc_julian_unrepeated[tv_del_ind .+ 1].-t0,
    1e6*86400*tdelay_v1[tv_del_ind](),
    label="predicted",
    marker=:xcross
)
scatter!(
    tv_jpl_utc_julian_unrepeated[tv_del_ind .+ 1].-t0,
    Float64.(jpl_radar[del_ind,3]),
    label="observed (JPL)",
    legend=:bottomright,
    marker=:cross,
    yerror=Float64.(jpl_radar[del_ind,4])
)
title!("Time delay vs time")
xlabel!("t-t0 [Julian days]")
ylabel!("Total time delay [microseconds]")

In [ ]:
scatter(
    tv_jpl_utc_julian_unrepeated[tv_dop_ind .+ 1].-t0,
    dshift_v1[tv_dop_ind](),
    label="predicted",
    marker=:xcross
)
scatter!(
    tv_jpl_utc_julian_unrepeated[tv_dop_ind .+ 1].-t0,
    Float64.(jpl_radar[dop_ind,3]),
    label="observed (JPL)",
    legend=:topright,
    marker=:cross,
    yerror=Float64.(jpl_radar[dop_ind,4])
)
title!("Doppler shift vs time")
xlabel!("t-t0 [Julian days]")
ylabel!("Total Doppler shift [Hz]")
#xlims!(1600,1640)

In [ ]:
scatter(
    tv_jpl_utc_julian_unrepeated[tv_del_ind .+ 1].-t0,
    residual_td(),
    yerror=Float64.(jpl_radar[del_ind,4]),
    marker=:cross
)
title!("Time delay O-C residuals")
xlabel!("t-t0 [Julian days]")
ylabel!("Total time delay resid. (O-C) [us]")

In [ ]:
scatter(
    tv_jpl_utc_julian_unrepeated[tv_dop_ind .+ 1].-t0,
    residual_ds(),
    yerror=Float64.(jpl_radar[dop_ind,4]),
    marker=:cross
)
title!("Doppler shift residuals")
xlabel!("t-t0 [Julian days]")
ylabel!("Total Doppler shift resid. (O-C) [Hz]")

In [ ]:
scatter(
    tv_jpl_utc_julian_unrepeated[tv_del_ind .+ 1].-t0,
    rel_res_td(),
    yerror=Float64.(jpl_radar[del_ind,4])./Float64.(jpl_radar[del_ind,3]),
    label="Relative time delay residual",
    marker=:cross,
    legend=:topright
)
title!("Time delay relative residuals")
xlabel!("t-t0 [Julian days]")

In [ ]:
scatter(
    tv_jpl_utc_julian_unrepeated[tv_dop_ind .+ 1].-t0,
    rel_res_ds(),
    yerror=Float64.(jpl_radar[dop_ind,4])./Float64.(jpl_radar[dop_ind,3]),
    label="Relative Doppler shift residual",
    legend=:bottomright,
    marker=:cross
)
title!("Doppler shift relative residuals")
xlabel!("t-t0 [Julian days]")

# Yarkovsky $A_2$ coefficient estimation

In [ ]:
tdelay_v1[1] # τ(A2) = p(A2)

In [ ]:
tdelay_v1[1]-constant_term(tdelay_v1[1]) # δτ(A2) = τ(A2) - τ(A2=0) = p(A2) - τ(A2=0) = p[1]*A2+p[2]*A2^2+... ( p[0]=τ(A2=0) )

In [ ]:
inverse( tdelay_v1[1]-constant_term(tdelay_v1[1]) ) # A2(δτ) = q(δτ) = q[1]*δτ+q[2]*δτ^2+... (q[0]=0)

In [ ]:
td_us = 1e6*86400*tdelay_v1[tv_del_ind]
ds_Hz = dshift_v1[tv_dop_ind];

In [ ]:
A2_δτ_v = (  inverse.( td_us-td_us() )  ) # vector of A2(δr) polynomials at each delay observation;
A2_δf_v = (  inverse.( ds_Hz-ds_Hz() )  ) # vector of A2(δvr) polynomials at each Doppler observation;

In [ ]:
A2_del_v = map((x,y)->x(y), A2_δτ_v, residual_td()); # A2(δτ) polynomials evaluated at the O-C time delay residuals;
A2_dop_v = map((x,y)->x(y), A2_δf_v, residual_ds()); # A2(δf_Doppler) polynomials evaluated at the O-C Doppler shift residuals;

In [ ]:
A2_del_v

In [ ]:
A2_dop_v

In [ ]:
mean(A2_del_v), std(A2_del_v)

In [ ]:
mean(A2_dop_v), std(A2_dop_v)

In [ ]:
scatter(tv_jpl_utc_julian_unrepeated[tv_del_ind .+ 1].-t0, A2_del_v, leg=false, marker=:cross)
xlabel!("t-t0 [Julian days]")
ylabel!("A2(dR) x 10^14 [au/d^2]")
ylims!(-12,0)

In [ ]:
scatter(tv_jpl_utc_julian_unrepeated[tv_dop_ind .+ 1].-t0, A2_dop_v, leg=false, marker=:cross)
xlabel!("t-t0 [Julian days]")
ylabel!("A2(dVR) x 10^14 [au/d^2]")

# Intervals

In [ ]:
using IntervalArithmetic

In [ ]:
residual_r_interval = interval.(rv1()-(del[del_ind,6]+del[del_ind,7])/au, rv1()-(del[del_ind,6]-del[del_ind,7])/au)
residual_vr_interval = interval.(vrv1()-(dop[dop_ind,6]+dop[dop_ind,7])/au, vrv1()-(dop[dop_ind,6]-dop[dop_ind,7])/au);

In [ ]:
A2_del_v_interval = map((x,y)->x(y), A2_δr_v, residual_r_interval); # A2(δr) polynomials evaluated at the δr residuals;
A2_dop_v_interval = map((x,y)->x(y), A2_δvr_v, residual_vr_interval); # A2(δvr) polynomials evaluated at the δvr residuals;

In [ ]:
scatter(tv_neodys_obs[del_neodys_obs_ind].-t0, A2_del_v, yerror=radius.(A2_del_v_interval), leg=false, marker=:cross)

In [ ]:
scatter(tv_neodys_obs[dop_neodys_obs_ind].-t0, A2_dop_v, yerror=radius.(A2_dop_v_interval), leg=false, marker=:cross)